# Tugas 2 (Membuat Pipelines beserta deskripsi)

## Pertama kita melakukan proses untuk mengambil data dari berbagai database, disini kita menggunakan dataset iris yang nantinya diambil 4 fitur dan 1 targetnya yaitu sepallength, sepalwidth, petallength, petalwidth, dan class. 

### SQL Server (petalwidth,class)

In [ ]:
# connect to SQL Server and read data from table iris.arff
import pyodbc
import pandas as pd

server = 'LAPTOP-H7SNH2OP\SQLEXPRESS' 
database = 'prosainsdata200411100066' 

cnxn = pyodbc.connect(r'Driver=ODBC Driver 17 for SQL Server;Server=LAPTOP-H7SNH2OP\SQLEXPRESS;Database=prosainsdata(200411100066);Trusted_Connection=yes;')  
cursor = cnxn.cursor()

# select 26 rows from SQL table to insert in dataframe.
query = "SELECT petalwidth,class FROM [prosainsdata(200411100066)].[dbo].[iris.arff]"
pwcs = pd.read_sql(query, cnxn)
pwcs = pd.DataFrame(pwcs, columns=['petalwidth','class'])
pwcs.head()

C:\Users\Dio\AppData\Local\Temp/ipykernel_18780/1868463945.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pwcs = pd.read_sql(query, cnxn)


,petalwidth,class
0,0.2,Iris-setosa
1,0.2,Iris-setosa
2,0.2,Iris-setosa
3,0.2,Iris-setosa
4,0.2,Iris-setosa


### Postgres (sepallength)

In [ ]:
# connect to PostgreSQL and read data from table iris.arff
import psycopg2

conn = psycopg2.connect(
    database="prosainsdata(200411100066)",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

cur.execute("SELECT sepallength FROM iris_arff")

rows = cur.fetchall()

sl = pd.DataFrame(rows, columns=['sepallength'])
sl.head()

,sepallength
0,5.1
1,4.9
2,4.7
3,4.6
4,5.0


### ElephantSQL (sepalwidth)

In [ ]:
# connect elephantsql and read data from table iris.arff
import psycopg2

conn = psycopg2.connect(
    database="mpbecome",
    user="mpbecome",
    password="qJrkkEHbBQhzOh5rwG0vL_T-6qVzLrJE",
    host="satao.db.elephantsql.com",
    port="5432"
)

cur = conn.cursor()

cur.execute("SELECT sepalwidth FROM iris")

rows = cur.fetchall()

sw = pd.DataFrame(rows, columns=['sepalwidth'])
sw.head()

,sepalwidth
0,3.5
1,3
2,3.2
3,3.1
4,3.6


### XAMPP (petallength)

In [ ]:
# connect to the localhost xaamp server
import mysql.connector
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='prosainsdata(200411100066)'
)
mycursor = mydb.cursor()

# select 26 rows from SQL table to insert in dataframe.
mycursor.execute("SELECT petallength FROM iris_arff")
myresult = mycursor.fetchall()

pl = pd.DataFrame(myresult, columns=['petallength'])
pl.head()

,petallength
0,1.4
1,1.4
2,1.3
3,1.5
4,1.4


### Setelah mengambil fitur dan target dari berbagai sumber maka langkah selanjutnya yaitu menggabungkan fitur dan target kedalam satu kolom

In [ ]:
iris_df = pd.concat([sl, sw, pl, pwcs], axis=1)
iris_df.head()

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Selanjutnya kita menyimpan per fitur untuk proses upload kedalam Orchest

In [ ]:
# convert to csv
pl.to_csv('pl_iris.csv', index=False)
sl.to_csv('sl_iris.csv', index=False)
sw.to_csv('sw_iris.csv', index=False)
pwcs.to_csv('pwcs_iris.csv', index=False)

## Selanjutnya, kita masuk kedalam Orchest untuk melakukan proses membuat pipeline

### import csv ke Orchest

### get_data.py

In [ ]:
import orchest
import pandas as pd
# from sklearn import datasets

# Explicitly cache the data in the "/data" directory since the
# kernel is running in a Docker container, which are stateless.
# The "/data" directory is a special directory managed by Orchest
# to allow data to be persisted and shared across pipelines and
# even projects.
# print("Dowloading California housing data...")
# data = datasets.fetch_california_housing(data_home="/data")
pl=pd.read_csv('dataset/pl_iris.csv')
sl=pd.read_csv('dataset/sl_iris.csv')
sw=pd.read_csv('dataset/sw_iris.csv')
pwcs=pd.read_csv('dataset/pwcs_iris.csv')

data = pd.concat([sl, sw, pl, pwcs], axis=1)

# Convert the data into a DataFrame.
# df_data = pd.DataFrame(data["data"], columns=data["feature_names"])
# df_target = pd.DataFrame(data["target"], columns=["MedHouseVal"])
# fitur
df_data = data.iloc[:, :-1]
# target
df_target = data.iloc[:, 4]

# Output the housing data so the next steps can retrieve it.
print("Outputting converted ")
orchest.output((df_data, df_target), name="data")
print("Success!")

data.head()

### split_data

In [ ]:
import orchest
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import MinMaxScaler
data = orchest.get_inputs() 
X, y = data["data"]
# scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
orchest.output((X_train, y_train, X_test, y_test), name="training_data")

### knn

In [ ]:
import numpy as np
import orchest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
# Retrieve the data from the previous step.
data = orchest.get_inputs()
X_train, y_train, X_test, y_test = data["training_data"]

model = KNeighborsClassifier(n_neighbors = 15)
model.fit(X_train, y_train)
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
test_accracy = accuracy_score(y_test, y_pred)
orchest.output(test_accracy, name="KNeighborsClassifier")

### naive_bayes

In [ ]:
import numpy as np
import orchest
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error
# Retrieve the data from the previous step.
data = orchest.get_inputs()
X_train, y_train, X_test, y_test = data["training_data"]

model = GaussianNB()
model.fit(X_train, y_train)
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
test_accracy = accuracy_score(y_test, y_pred)
orchest.output(test_accracy, name="GaussianNB")

### logistigreq

In [ ]:
import numpy as np
import orchest
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
# Retrieve the data from the previous step.
data = orchest.get_inputs()
X_train, y_train, X_test, y_test = data["training_data"]

model = LogisticRegression()
model.fit(X_train, y_train)
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
test_accracy = accuracy_score(y_test, y_pred)
orchest.output(test_accracy, name="logistic-regression-accuracy")

### collect-result

In [ ]:
import orchest
data = orchest.get_inputs()
for name, value in data.items():
    if name != "unnamed":
        print(f"\n{name:30} {value}")

## Setelah melakukan proses diatas maka langkah yang terakhir adalah dengan mendapat hasil dari perhitungan dalam pipeline, didalam pipeline saya terdapat 3 model yang digunakan untuk klasifikasi